In [1]:
!nvidia-smi

Thu Aug 18 09:38:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Imports

In [2]:
%%capture
!pip install transformers
!pip install pytorch-lightning

In [3]:
import gc
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from google.colab import drive
import re

import random
import torch

def set_seed(seed=0):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

import warnings
warnings.simplefilter('ignore')

In [4]:
drive.mount('/content/drive', force_remount=False)

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/signate_student_cup_2022
train_path = './data/train.csv'
test_path = './data/test.csv'
submit_path = './data/submit_sample.csv'

/content/drive/MyDrive/signate_student_cup_2022


In [6]:
job_flags = ['Data scientist', 'Machine learning engineer','Software engineer','Consultant']

In [7]:
def text_cleaning(texts):
    clean_texts = []
    num_lines = []
    num_words = []
    words_chunk = []
    for text in texts:
        clean_lines = []
        # print(text, "\n")
        lines = text.split(r"</li>")
        for line in lines:
            line = remove_tag(line)
            # print(line)
            #バックスラッシュをスペースに置き換え
            clean_line = re.sub(r'[\\]', '', line)
            # clean_line = re.sub(r'[/]', ' and ', line)# test
            # print(clean_line)
            clean_line = clean_line.strip()
            # print(clean_line, "\n")
            clean_line = clean_line + ('' if clean_line.endswith('.') else '.')
            if len(clean_line)!=1:
                # print(clean_line, "\n")
                clean_lines.append(clean_line)
        # print(clean_lines)
        clean_texts.append(' '.join(clean_lines))
        
        num_lines.append(len(clean_lines))
        num_word = len(str(clean_lines).split())
        num_words.append(num_word)
        word_chunk = ((num_word-1)//240)+1
        words_chunk.append(word_chunk)

    return clean_texts, num_lines, num_words, words_chunk

def remove_tag(x):
    p = re.compile(r"<[^>]*?>")
    return p.sub('',x)

In [8]:
job_flags = ['Data scientist', 'Machine learning engineer','Software engineer','Consultant']

train_data = pd.read_csv(train_path)
train_data['description'], num_lines, num_words, words_chunk = text_cleaning(train_data['description'])
for i in range(3):
  print("===== ", i, " =====")
  for j in range(10):
    temp_df = train_data[train_data["jobflag"]==i+1].reset_index(drop=True)
    print(temp_df.loc[j, "description"], "\n")

=====  0  =====
Consolidate dashboards across the team and help to drive interpretation, socialization and adoption of operational sales reports. 

Maintain and improve existing predictive models to evaluate media efficacy and effectiveness, willing to develop new models to help answer business questions. Glean actionable insights to support in-market campaign optimization. Support proactive "what-if" analysis and guide marketing and advertising decision-making. Collaborate and partner with other agency departments -- Media, Brand Planning, PR/Social Influence,. 

Participate in communication of research methods, implementation, and results to varied audience of clinicians, scientists, analysts, and programmers. Contribute to manuscript writing for results publication, authors abstracts, and presents at professional conferences. Contribute to experiments and summaries for grant proposal preparation. 

Implement process improvements and automation for efficiency where appropriate. Desig

In [9]:
train_data = pd.read_csv(train_path)
print(train_data["jobflag"].value_counts())
train_data = train_data[train_data["jobflag"]!=4].reset_index(drop=True)
print(train_data["jobflag"].value_counts())
train_data['description'], num_lines, num_words, words_chunk = text_cleaning(train_data['description'])
train_data['num_lines'] = num_lines
train_data['num_words'] = num_words
train_data['words_chunk'] = words_chunk
train_data['jobflag'] = train_data['jobflag']-1
train_data.head(20)

4    505
1    468
3    455
2     88
Name: jobflag, dtype: int64
1    468
3    455
2     88
Name: jobflag, dtype: int64


,id,description,jobflag,num_lines,num_words,words_chunk
0,0,Develop cutting-edge web applications that per...,2,8,112,1
1,1,"Designs and develops high quality, scalable an...",2,15,221,1
2,3,"Work on the technical design, development, rel...",2,5,79,1
3,5,Participates in standard business and technica...,2,4,49,1
4,8,Consolidate dashboards across the team and hel...,0,1,17,1
5,9,Maintain and improve existing predictive model...,0,4,53,1
6,10,Optimize deep learning frameworks like TensorF...,1,5,64,1
7,11,Participate in communication of research metho...,0,3,41,1
8,13,"Research, prototype, identify, and build predi...",1,3,39,1
9,14,System architecture - collaborating with senio...,2,6,72,1


In [10]:
train_data.shape

(1011, 6)

In [11]:
test_data = pd.read_csv(test_path)
test_data['description'], num_lines, num_words, words_chunk = text_cleaning(test_data['description'])
test_data['num_lines'] = num_lines
test_data['num_words'] = num_words
test_data['words_chunk'] = words_chunk
test_data.head(5)

,id,description,num_lines,num_words,words_chunk
0,1516,Building decision-making models and proposing ...,2,30,1
1,1517,Educate homeowners on the benefits of solar en...,5,43,1
2,1518,"Design, develop, document, and implement web a...",8,82,1
3,1519,Apply advanced technical expertise and skills ...,6,81,1
4,1520,Project manage and deliver against our roadmap...,4,35,1


In [12]:
submit_df = pd.read_csv(submit_path)
submit_df.head(5)

,1516,1
0,1517,1
1,1518,1
2,1519,1
3,1520,1
4,1521,1


# Dataset

In [13]:
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer
import pytorch_lightning as pl
# pl.seed_everything(0, workers=True)
from torch.utils.data import DataLoader
from transformers import AutoModel, AdamW, get_cosine_schedule_with_warmup
import torch.nn as nn
import math
import torch.nn.functional as F
from torchmetrics import F1Score
from transformers import AutoConfig

from torchmetrics.functional import precision_recall
from sklearn.metrics import confusion_matrix

In [14]:
class Careers_Dataset(Dataset):

  def __init__(self, data):
    self.data = data
    
  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    item = self.data.iloc[index]
    return item


# Classification

In [15]:
from transformers import pipeline
# pipe = pipeline("fill-mask", model="bert-base-uncased")
# pipe = pipeline("zero-shot-classification", device=0)# best
pipe = pipeline("zero-shot-classification", model="facebook/bart-large-mnli", device=0)

## zero-shot test

In [16]:
# tokenizer = AutoTokenizer.from_pretrained("zero-shot-classification")
true_labels = ['Data scientist', 'Machine learning engineer','Software engineer','Consultant']
all_labels = ['Data scientist', 'Machine learning engineer','Software engineer']
label_mapping = {'Data scientist': 0, 'Machine learning engineer': 1,'Software engineer': 2}

ds = Careers_Dataset(train_data)

for i in range(3):

  sample = ds[i]
  print(f"Labels: {sample['jobflag']} >> {true_labels[sample['jobflag']]}")

  if sample['jobflag'] == 3:
    print(sample["description"])
    print("\n ===============================================")
    continue

  output = pipe(sample["description"], all_labels)
  print(output["labels"][0])
  print(label_mapping[output["labels"][0]])
  # print(output["sequence"][:400])
  print("[Predictions]")
  for label, score in zip(output["labels"], output["scores"]):
      print(f"{label}, {score:.2f}")

  print("\n ===============================================")

# 特に不均衡データにはzero-shotありか... consultant抜きで考えてみる

Labels: 2 >> Software engineer
Software engineer
2
[Predictions]
Software engineer, 0.60
Machine learning engineer, 0.22
Data scientist, 0.18

Labels: 2 >> Software engineer
Software engineer
2
[Predictions]
Software engineer, 0.77
Machine learning engineer, 0.12
Data scientist, 0.11

Labels: 2 >> Software engineer
Software engineer
2
[Predictions]
Software engineer, 0.71
Data scientist, 0.15
Machine learning engineer, 0.14



In [17]:
# tokenizer = AutoTokenizer.from_pretrained("zero-shot-classification")
true_labels = ['Data scientist', 'Machine learning engineer','Software engineer','Consultant']
all_labels = ['Data scientist', 'Machine learning engineer','Software engineer', 'Adviser']

ds = Careers_Dataset(train_data)

for i in range(20):

  sample = ds[i]
  print(f"Labels: {sample['jobflag']} >> {true_labels[sample['jobflag']]}")

  output = pipe(sample["description"], all_labels)
  # print(output["sequence"][:400])
  print("[Predictions]")
  for label, score in zip(output["labels"], output["scores"]):
      print(f"{label}, {score:.2f}")

  print("\n ===============================================")

# 特に不均衡データにはzero-shotありか... consultant抜きで考えてみる

Labels: 2 >> Software engineer
[Predictions]
Software engineer, 0.40
Adviser, 0.34
Machine learning engineer, 0.14
Data scientist, 0.12

Labels: 2 >> Software engineer
[Predictions]
Adviser, 0.60
Software engineer, 0.30
Machine learning engineer, 0.05
Data scientist, 0.04

Labels: 2 >> Software engineer
[Predictions]
Adviser, 0.50
Software engineer, 0.36
Data scientist, 0.08
Machine learning engineer, 0.07

Labels: 2 >> Software engineer
[Predictions]
Adviser, 0.48
Software engineer, 0.26
Machine learning engineer, 0.13
Data scientist, 0.13

Labels: 0 >> Data scientist
[Predictions]
Adviser, 0.48
Data scientist, 0.25
Software engineer, 0.16
Machine learning engineer, 0.11

Labels: 0 >> Data scientist
[Predictions]
Adviser, 0.34
Data scientist, 0.28
Machine learning engineer, 0.25
Software engineer, 0.14

Labels: 1 >> Machine learning engineer
[Predictions]
Machine learning engineer, 0.46
Software engineer, 0.28
Data scientist, 0.15
Adviser, 0.12

Labels: 0 >> Data scientist
[Prediction

## zero-shot predicting

In [18]:
# Zero-shotだとConsultantに過敏に反応: 3種類でやってみる → zero-shotがMachine learning engineerと予測したとこだけ入れ替える??
# binary分類でMachine learning engineerだけはやる?? <> Debartaは3or4種類分類

In [19]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 66.9 MB/s 
     |████████████████████████████████| 115 kB 73.4 MB/s 
     |████████████████████████████████| 127 kB 71.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [20]:
from datasets import Dataset

"""
all_labels = ['Data scientist', 'Machine learning engineer','Software engineer']
label_mapping = {'Data scientist': 0, 'Machine learning engineer': 1,'Software engineer': 2}
"""

"""
all_labels = ['Data scientist', 'AI engineer','Software engineer']
label_mapping = {'Data scientist': 0, 'AI engineer': 1,'Software engineer': 2}
"""

"""
all_labels = ['Data Science', 'Artificial Intelligence', 'Software']
label_mapping = {'Data Science': 0, 'Artificial Intelligence': 1,'Software': 2}
"""

# tensor(0.6898)  [0.7573033707865169, 0.5, 0.8122362869198311]
"""
Data scientist 
[[458  85]
 [131 337]]
Precision:  0.7985781990521327
Recall:  0.7200854700854701

 Machine learning engineer 
[[873  50]
 [ 42  46]]
Precision:  0.4791666666666667
Recall:  0.5227272727272727

 Software engineer 
[[448 108]
 [ 70 385]]
Precision:  0.7809330628803245
Recall:  0.8461538461538461
all_labels = ['Data Analysis', 'Machine Learning Engineering', 'Software Engineering']
label_mapping = {'Data Analysis': 0, 'Machine Learning Engineering': 1,'Software Engineering': 2}
"""
# TN FP
# FN TP

all_labels = ['Data Analysis', 'Machine Learning Engineering', 'Software Engineering']
label_mapping = {'Data Analysis': 0, 'Machine Learning Engineering': 1,'Software Engineering': 2}


# all_labels = ['Data Analysis', 'AI', 'Software Engineering']
# label_mapping = {'Data Analysis': 0, 'AI': 1,'Software Engineering': 2}

def zero_shot_pipeline(example):
    output = pipe(example["description"], all_labels)
    example["predicted_labels"] = output["labels"]
    example["scores"] = output["scores"]
    return example

ds = Dataset.from_pandas(train_data.reset_index(drop=True))
ds_zero_shot = ds.map(zero_shot_pipeline)

  0%|          | 0/1011 [00:00<?, ?ex/s]

In [21]:
ds_zero_shot

Dataset({
    features: ['id', 'description', 'jobflag', 'num_lines', 'num_words', 'words_chunk', 'predicted_labels', 'scores'],
    num_rows: 1011
})

In [22]:
def get_preds(example, threshold=None, topk=None):
    return {"predicted_ids": label_mapping[example["predicted_labels"][0]]}

In [23]:
ds_zero_shot = ds_zero_shot.map(get_preds, batched=False)

  0%|          | 0/1011 [00:00<?, ?ex/s]

In [24]:
ds_zero_shot

Dataset({
    features: ['id', 'description', 'jobflag', 'num_lines', 'num_words', 'words_chunk', 'predicted_labels', 'scores', 'predicted_ids'],
    num_rows: 1011
})

In [25]:
from torchmetrics import F1Score
f1_func = F1Score(num_classes=4, average="macro")

F1_labels = ["DS", "ML", "SE"]
def get_F1_score(ds):
    y_true = torch.tensor(ds["jobflag"])
    y_pred = torch.tensor(ds["predicted_ids"])

    print(len(y_true))

    for t in range(3):
        print(f'[{F1_labels[t]}]')
        for p in range(3):
            if t==p:
                # 正解
                print(f'● pred={F1_labels[p]} >> {(sum((y_true==t)&(y_pred==p)) / sum(y_true==t))*100 :.2f}')
            else:
                # 不正解
                print(f' × pred={F1_labels[p]} >> {sum((y_true==t)&(y_pred==p)) / sum(y_true==t)*100 :.2f}')
        print("\n")
    return f1_func(y_pred, y_true)# 実装

def get_each_F1_score(ds):
    labels = np.array(ds["jobflag"])
    predicted = np.array(ds["predicted_ids"])

    label_f1_scores = []
    for l in range(3):
        print("\n", ['Data scientist', 'Machine learning engineer','Software engineer'][l], "\n")
        temp_predicted = np.where(predicted==l, 1, 0)
        temp_labels = np.where(labels==l, 1, 0)
        con_matrix = confusion_matrix(temp_labels, temp_predicted)
        print(con_matrix)
        TN = con_matrix[0][0]
        FP = con_matrix[0][1]
        FN = con_matrix[1][0]
        TP = con_matrix[1][1]
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        print("Precision: ", precision)
        print("Recall: ", recall)
        temp_f1 = (2*precision*recall) / (precision+recall)
        label_f1_scores.append(temp_f1)

        fn_point = (temp_predicted==0) == (temp_labels==1)
        # print("FN: 偽陰性")
        for i in range(10):
            # print("\n    ", np.array(ds["description"])[fn_point][i])
            pass

        fp_point = (temp_predicted==1) == (temp_labels==0)
        # print("FP: 偽陽性")
        for i in range(10):
            # print("\n    ", np.array(ds["description"])[fp_point][i])
            pass

    return label_f1_scores# 実装

In [26]:
f1_score = get_F1_score(ds_zero_shot)
f1_score

1011
[DS]
● pred=DS >> 72.01
 × pred=ML >> 9.83
 × pred=SE >> 18.16


[ML]
 × pred=DS >> 21.59
● pred=ML >> 52.27
 × pred=SE >> 26.14


[SE]
 × pred=DS >> 14.51
 × pred=ML >> 0.88
● pred=SE >> 84.62




tensor(0.6898)

In [27]:
# ['Data scientist', 'Machine learning engineer','Software engineer']
each_f1_score = get_each_F1_score(ds_zero_shot)
each_f1_score
# TN FP
# FN TP


 Data scientist 

[[458  85]
 [131 337]]
Precision:  0.7985781990521327
Recall:  0.7200854700854701

 Machine learning engineer 

[[873  50]
 [ 42  46]]
Precision:  0.4791666666666667
Recall:  0.5227272727272727

 Software engineer 

[[448 108]
 [ 70 385]]
Precision:  0.7809330628803245
Recall:  0.8461538461538461


[0.7573033707865169, 0.5, 0.8122362869198311]

In [28]:
for desc in train_data[train_data["jobflag"]==1]["description"].reset_index(drop=True):
    print(desc, "\n")

Optimize deep learning frameworks like TensorFlow, PyTorch, etc. on AMD GPUs in upstream open-source repositories. Collaborate and interact with internal GPU library teams to analyze and optimize training and inference for deep learning. Work in a distributed computing setting to optimize for both scale-up (multi-GPU) and scale-out (multi-node) systems. Work with cutting-edge compiler technologies. Optimize the entire deep learning pipeline including graph compiler integration. 

Research, prototype, identify, and build predictive products. You will write tests to ensure the robustness and reliability of your productionized models. You will guide and mentor team members and help educate the business on the art of the possible. 

Optimizing our ML model and methods for determining Liveness/Realness of human faces. Delivering optimized models to the app development team for integration in the apps. Communication with both internal and 3rd party technical teams who wish to integrate our m

In [29]:
ds_zero_shot

Dataset({
    features: ['id', 'description', 'jobflag', 'num_lines', 'num_words', 'words_chunk', 'predicted_labels', 'scores', 'predicted_ids'],
    num_rows: 1011
})

In [30]:
F1_labels = ["DS", "ML", "SE"]
for example in ds_zero_shot:
    for p in range(3):
        if example['jobflag']==1 and example['predicted_ids']==p:
            print(f'pred={F1_labels[p]}')
            print(example['description'], "\n")

pred=ML
Optimize deep learning frameworks like TensorFlow, PyTorch, etc. on AMD GPUs in upstream open-source repositories. Collaborate and interact with internal GPU library teams to analyze and optimize training and inference for deep learning. Work in a distributed computing setting to optimize for both scale-up (multi-GPU) and scale-out (multi-node) systems. Work with cutting-edge compiler technologies. Optimize the entire deep learning pipeline including graph compiler integration. 

pred=ML
Research, prototype, identify, and build predictive products. You will write tests to ensure the robustness and reliability of your productionized models. You will guide and mentor team members and help educate the business on the art of the possible. 

pred=ML
Optimizing our ML model and methods for determining Liveness/Realness of human faces. Delivering optimized models to the app development team for integration in the apps. Communication with both internal and 3rd party technical teams who